In [2]:
import pandas as pd

In [19]:
import pywikibot
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

In [4]:
!pip install xlrd

  Using cached https://files.pythonhosted.org/packages/07/e6/e95c4eec6221bfd8528bcc4ea252a850bffcc4be88ebc367e23a1a84b0bb/xlrd-1.1.0-py2.py3-none-any.whl


In [13]:
!pip install XlsxWriter

    100% |████████████████████████████████| 143kB 1.5MB/s eta 0:00:01


In [22]:
def create_item(site, label_dict):
    new_item = pywikibot.ItemPage(site)
    new_item.editLabels(labels=label_dict, summary="Creating Red List item")
    return new_item

In [23]:
def set_claim(item, property, string_data):
    claim = pywikibot.Claim(repo, property)
    claim.setTarget(string_data)
    item.addClaim(claim, bot=False, summary="Test")
    return claim

In [24]:
red_list_df = pd.read_excel("red2.xlsx")

In [11]:
red_list_df['Wikidata ID'].fillna('---', inplace=True)

In [25]:
red_list_df.head()

,Number,Nombre,Artículo en español,Wikidata ID,Latitud,Longitud,Localidad,Municipio,Provincia,Comunidad autónoma,Descripción,URL,Estatus,Columna1,Columna2,Unnamed: 15
0,1,Acueducto de Albatana,Acueducto de Albatana,Q4782853,38.583449,-1.545564,NaN,Albatana,Albacete,Castilla-La Mancha,Acueducto del siglo XIX.,http://listarojapatrimonio.org/ficha/acueducto...,Done,NaN,NaN,NaN
1,2,Alcazaba de Baza,NaN,NaN,37.490234,-2.775614,NaN,Baza,Granada,Andalucía,"Fortaleza árabe de gran extensión, con el 70-8...",http://listarojapatrimonio.org/ficha/alcazaba-...,Reviewed,NaN,NaN,NaN
2,3,Alcázar de Toro,NaN,Q5664259,41.527442,-5.391337,NaN,Toro,Zamora,Castilla y León,"Edificación defensiva de planta cuadrada, cuya...",http://listarojapatrimonio.org/ficha/alcazar-d...,NaN,NaN,NaN,NaN
3,4,Alcubillas de Córdoba,NaN,NaN,37.891789,-4.779423,NaN,Córdoba,Córdoba,Andalucía,Depósitos de captación y distribución de aguas...,http://listarojapatrimonio.org/ficha/alcubilla...,Reviewed,NaN,NaN,NaN
4,5,Aljibe de la Casa de las Veletas,NaN,NaN,39.472755,-6.370343,NaN,Cáceres,Cáceres,Extremadura,Aljibe del siglo XIII de origen árabe,http://listarojapatrimonio.org/ficha/aljibe-hi...,Reviewed,NaN,"File:Palacio de las Veletas, aljibe, Cáceres.JPG",NaN


In [34]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if index > 1:
        break
    if pd.isnull(row['Wikidata ID']):
        labels = {"es": row['Nombre']}
        print (labels)
        new_item = create_item(site, labels)
        new_item_id = new_item.getID()
        new_claim = (new_item, 'P4868', row)
        row['Wikidata ID'] = new_item_id
    output_df = output_df.append(row, ignore_index=True)

{'es': 'Acueducto de Albatana'}
<class 'float'>
{'es': 'Alcazaba de Baza'}


In [18]:
writer = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Sheet1')
writer.save()